# Setup

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_config = {
    'file_path':'data/',
    'pce_fname':'PCEBridge.xlsx', # CPI makeup
    'mkt_share_fname':'market_share.xlsx', # D matrix
    'dom_comm_use_fname':'domestic_comm_use.xlsx', # B^d matrix
    'gross_output_fname':'GrossOutput.xlsx', # used to calc v, m, delta P
    'int_in_fname':'IntermediateInputs.xlsx', # M
    'val_add_fname':'ValueAdded.xlsx' # V
}
time_config = {
    'start_year':1997, # inclusive
    'end_year':2021 # inclusive
}

# Read Data

In [3]:
def read_mkt_share(year):
    # read in market share matrix for input year 
    # this is fraction of each commodities' total output produced by each industry
    df = pd.read_excel(
        io=data_config['file_path']+data_config['mkt_share_fname'],
        sheet_name=str(year),
        header=[5,6],
        index_col=[0,1]
    )
    df.dropna(inplace=True)
    return df

def read_mkt_share_all(time_config):
    # read all market share matrix for all years in time config
    # this is fraction of each commodities' total output produced by each industry   
    mkt_share_dfs = {}
    for time in time_config:
        df = read_mkt_share(year)
        mkt_share_dfs[time] = df
    return mkt_share_dfs

In [4]:
test_mkt_share = read_mkt_share(2021)
test_mkt_share

,Industries/Commodities,111CA,113FF,211,212,213,22,23,321,327,331,...,721,722,81,GFGD,GFGN,GFE,GSLG,GSLE,Used,Other
,Industry Description,Farms,"Forestry, fishing, and related activities",Oil and gas extraction,"Mining, except oil and gas",Support activities for mining,Utilities,Construction,Wood products,Nonmetallic mineral products,Primary metals,...,Accommodation,Food services and drinking places,"Other services, except government",Federal general government (defense),Federal general government (nondefense),Federal government enterprises,State and local general government,State and local government enterprises,"Scrap, used and secondhand goods",Noncomparable imports and rest-of-the-world adjustment
111CA,Farms,0.999560,0.067274,0.000000,0.000000,0.000000,0.000000,0.0,0.000067,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
113FF,"Forestry, fishing, and related activities",0.000076,0.881106,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
211,Oil and gas extraction,0.000000,0.000000,0.999369,0.000880,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
212,"Mining, except oil and gas",0.000000,0.000000,0.000000,0.987722,0.001713,0.000000,0.0,0.000000,0.003267,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
213,Support activities for mining,0.000000,0.000000,0.000000,0.000856,0.998277,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GFGD,Federal general government (defense),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0
GFGN,Federal general government (nondefense),0.000000,0.000000,0.000000,0.000000,0.000000,0.000098,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.000000,0.000000,1.0
GFE,Federal government enterprises,0.000000,0.000000,0.000000,0.000000,0.000000,0.018339,0.0,0.000000,0.000000,0.0,...,0.000000,0.001197,0.000000,0.0,0.0,1.0,0.0,0.000000,0.000000,0.0


In [5]:
# testing multiindex selection
# test_mkt_share.loc[test_mkt_share.index.get_level_values(0).isin(ind_map),:]
# test_mkt_share.loc[:,test_mkt_share.columns.get_level_values(0).isin(comm_map)]


In [6]:
def read_dom_use(year):
    # read in domestic comm use matrix for input year 
    df = pd.read_excel(
        io=data_config['file_path']+data_config['dom_comm_use_fname'],
        sheet_name=str(year),
        header=[5,6],
        index_col=[0,1]
    )
    return df

def read_dom_use_all(time_config):
    # read all domestict comm use matrix for all years in time config
    dom_use_dfs = {}
    for time in time_config:
        df = read_dom_use(time)
        dom_use_dfs[time] = df
    return dom_use_dfs

In [7]:
test_dom_use = read_dom_use(2021)
test_dom_use

,Commodities/Industries,111CA,113FF,211,212,213,22,23,321,327,331,...,711AS,713,721,722,81,GFGD,GFGN,GFE,GSLG,GSLE
,Commodity Description,Farms,"Forestry, fishing, and related activities",Oil and gas extraction,"Mining, except oil and gas",Support activities for mining,Utilities,Construction,Wood products,Nonmetallic mineral products,Primary metals,...,"Performing arts, spectator sports, museums, and related activities","Amusements, gambling, and recreation industries",Accommodation,Food services and drinking places,"Other services, except government",Federal general government (defense),Federal general government (nondefense),Federal government enterprises,State and local general government,State and local government enterprises
111CA,Farms,0.173553,0.003526,0.000000,0.000511,0.000208,0.000000,0.001356,0.000000,0.000000,0.000000,...,0.000000,0.002113,0.000155,0.000698,0.000014,0.000003,0.000573,0.000000,0.000700,0.000000
113FF,"Forestry, fishing, and related activities",0.052062,0.144284,0.000000,0.000741,0.000000,0.000000,0.000000,0.097674,0.000001,0.000000,...,0.000000,0.000107,0.000144,0.000329,0.000011,0.000000,0.000000,0.000013,0.001209,0.000000
211,Oil and gas extraction,0.000000,0.000000,0.103113,0.000083,0.000009,0.044571,0.000000,0.000006,0.000002,0.000063,...,0.000003,0.000064,0.000157,0.000077,0.000005,0.000000,0.000000,0.006025,0.000000,0.044277
212,"Mining, except oil and gas",0.003139,0.000005,0.000128,0.079912,0.000344,0.009573,0.010456,0.000009,0.046471,0.037799,...,0.001365,0.002593,0.000275,0.000225,0.000764,0.000000,0.001053,0.000000,0.000330,0.008645
213,Support activities for mining,0.000000,0.000000,0.020004,0.018675,0.010956,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Other,Noncomparable imports and rest-of-the-world adjustment,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
V001,Compensation of employees,0.067880,0.496264,0.017889,0.113505,0.374724,0.142098,0.337109,0.188588,0.226096,0.128616,...,0.321720,0.548203,0.245857,0.289591,0.514410,0.390153,0.368424,0.706385,0.581872,0.313837
V002,"Taxes on production and imports, less subsidies",-0.031005,0.017558,0.088914,0.059465,-0.001070,0.110436,-0.010530,-0.000598,0.006626,0.007833,...,-0.035355,0.096534,0.076055,0.006220,0.018094,0.000000,0.000000,-0.101775,0.000000,-0.100576


In [8]:
def read_gross_output():
    df = pd.read_excel(
        io=data_config['file_path']+data_config['gross_output_fname'],
        sheet_name='TGO105-A',
        header=[7],
        index_col=[0,1]
    )
    df.drop(columns=df.columns[0], axis=1, inplace=True)
    return df

In [9]:
test_gross = read_gross_output()
test_gross

,,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Line,,,,,,,,,,,,,,,,,,,,,,
1,All industries,15393320.0,16216785.0,17270727.0,18625220.0,18881157.0,19170837.0,20138014.0,21688905.0,23514740.0,24924662.0,...,29222821.0,30350112.0,31756413.0,32183068.0,32855063.0,34392092.0,36489532.0,37709701.0,36561975.0,41404764.0
2,Private industries,13728169.0,14487020.0,15436428.0,16674534.0,16801227.0,16955232.0,17795467.0,19210000.0,20892146.0,22147977.0,...,25833601.0,26923094.0,28264017.0,28616198.0,29205311.0,30647259.0,32553709.0,33604067.0,32393106.0,36937182.0
3,"Agriculture, forestry, fishing, and hunting",257882.0,249763.0,245315.0,243596.0,251356.0,241873.0,269299.0,308554.0,300187.0,304036.0,...,451069.0,486245.0,497416.0,456433.0,432253.0,452386.0,451330.0,439022.0,447779.0,526044.0
4,Farms,219380.0,207822.0,199949.0,204313.0,212334.0,202363.0,227873.0,263177.0,253552.0,256245.0,...,404166.0,436934.0,441667.0,402908.0,379677.0,395379.0,395102.0,387890.0,394395.0,466636.0
5,"Forestry, fishing, and related activities",38502.0,41942.0,45365.0,39283.0,39022.0,39509.0,41426.0,45377.0,46636.0,47792.0,...,46902.0,49312.0,55750.0,53526.0,52575.0,57006.0,56228.0,51132.0,53384.0,59408.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Information-communications-technology-producing industries\3\,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1. Consists of agriculture, forestry, fishing, and hunting; mining; construction; and manufacturing.",Information-communications-technology-producing industries\3\,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2. Consists of utilities; wholesale trade; retail trade; transportation and warehousing; information; finance, insurance, real estate, rental, and leasing; professional and business services; educational services, health care, and social assistance; arts, entertainment, recreation, accommodation, and food services; and other services, except government.",Information-communications-technology-producing industries\3\,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
test_gross.tail(10)

1997  \
Line                                                                                                               
97                                                     Government enterprises                           130099.0   
                                                     Addenda:                                                NaN   
98                                                       Private goods-producing industries\1\         5041816.0   
99                                                       Private services-producing industries\2\      8686353.0   
100                                                      Information-communications-technology-pro...   993709.0   
                                                         Information-communications-technology-pro...        NaN   
1. Consists of agriculture, forestry, fishing, ...       Information-communications-technology-pro...        NaN   
2. Consists of utilities; wholesale trade; reta...       Information-communications-technology-pro...        NaN   
3. Consists of computer and electronic product ...       Information-communications-technology-pro...        NaN   
Note. Detail may not add to total due to rounding.       Information-communications-technology-pro...        NaN   

                                                                                                            1998  \
Line                                                                                                               
97                                                     Government enterprises                           134784.0   
                                                     Addenda:                                                NaN   
98                                                       Private goods-producing industries\1\         5143366.0   
99                                                       Private services-producing industries\2\      9343654.0   
100                                                      Information-communications-technology-pro...  1087193.0   
                                                         Information-communications-technology-pro...        NaN   
1. Consists of agriculture, forestry, fishing, ...       Information-communications-technology-pro...        NaN   
2. Consists of utilities; wholesale trade; reta...       Information-communications-technology-pro...        NaN   
3. Consists of computer and electronic product ...       Information-communications-technology-pro...        NaN   
Note. Detail may not add to total due to rounding.       Information-communications-technology-pro...        NaN   

                                                                                                             1999  \
Line                                                                                                                
97                                                     Government enterprises                            141176.0   
                                                     Addenda:                                                 NaN   
98                                                       Private goods-producing industries\1\          5322819.0   
99                                                       Private services-producing industries\2\      10113609.0   
100                                                      Information-communications-technology-pro...   1224342.0   
                                                         Information-communications-technology-pro...         NaN   
1. Consists of agriculture, forestry, fishing, ...       Information-communications-technology-pro...         NaN   
2. Consists of utilities; wholesale trade; reta...       Information-communications-technology-pro...         NaN   
3. Consists of computer and electronic product ...       Information-communications-technology-pro...         NaN   
Note. Detail may not add to total d

In [11]:
def read_delta_gross_output():
    df = pd.read_excel(
        io=data_config['file_path']+data_config['gross_output_fname'],
        sheet_name='TGO107-A',
        header=[7],
        index_col=[0,1]
    )
    df.drop(columns=df.columns[0], axis=1, inplace=True)
    return df

In [12]:
test_delta_gross = read_delta_gross_output()
test_delta_gross

1998  \
Line                                                                                                          
1                                                      All industries                                   0.0   
2                                                  Private industries                                  -0.2   
3                                                    Agriculture, forestry, fishing, and hunting       -5.2   
4                                                      Farms                                           -5.8   
5                                                      Forestry, fishing, and related activities       -1.6   
...                                                                                                     ...   
100                                                      Information-communications-technology-pro...  -7.2   
                                                         Information-communications-technology-pro...   NaN   
1. Consists of agriculture, forestry, fishing, ...       Information-communications-technology-pro...   NaN   
2. Consists of utilities; wholesale trade; reta...       Information-communications-technology-pro...   NaN   
3. Consists of computer and electronic product ...       Information-communications-technology-pro...   NaN   

                                                                                                       1999  \
Line                                                                                                          
1                                                      All industries                                   1.3   
2                                                  Private industries                                   1.1   
3                                                    Agriculture, forestry, fishing, and hunting       -5.0   
4                                                      Farms                                           -5.9   
5                                                      Forestry, fishing, and related activities       -0.8   
...                                                                                                     ...   
100                                                      Information-communications-technology-pro...  -5.4   
                                                         Information-communications-technology-pro...   NaN   
1. Consists of agriculture, forestry, fishing, ...       Information-communications-technology-pro...   NaN   
2. Consists of utilities; wholesale trade; reta...       Information-communications-technology-pro...   NaN   
3. Consists of computer and electronic product ...       Information-communications-technology-pro...   NaN   

                                                                                                       2000  \
Line                                                                                                          
1                                                      All industries                                   3.2   
2                                                  Private industries                                   3.1   
3                                                    Agriculture, forestry, fishing, and hunting        0.7   
4                                                      Farms                                            0.9   
5                                                      Forestry, fishing, and related activities        0.1   
...                                                                                                     ...   
100                                                      Information-communications-technology-pro...  -4.2   
                                                         Information-communications-technology-pro...   NaN   
1. Consists of agriculture, forestry, fishing, ...       Information-communications-technology-pro...   N

In [13]:
def read_value_add():
    df = pd.read_excel(
        io=data_config['file_path']+data_config['val_add_fname'],
        sheet_name='TVA105-A',
        header=[7],
        index_col=[0,1]
    )
    df.drop(columns=df.columns[0], axis=1, inplace=True)
    return df

In [14]:
test_val_add = read_value_add()
test_val_add

,,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Line,,,,,,,,,,,,,,,,,,,,,,
1,Gross domestic product,8577552.0,9062817.0,9631172.0,10250952.0,10581929.0,10929108.0,11456450.0,12217196.0,13039197.0,13815583.0,...,16253970.0,16843196.0,17550687.0,18206023.0,18695106.0,19477337.0,20533058.0,21380976.0,21060474.0,23315081.0
2,Private industries,7431992.0,7871500.0,8378827.0,8927924.0,9188997.0,9454715.0,9904098.0,10585924.0,11328929.0,12023591.0,...,14094484.0,14630684.0,15279312.0,15866579.0,16310916.0,17031697.0,17987542.0,18762520.0,18360184.0,20502200.0
3,"Agriculture, forestry, fishing, and hunting",108637.0,99756.0,92602.0,98306.0,99830.0,95917.0,114597.0,143822.0,129452.0,126357.0,...,178689.0,214285.0,198912.0,180138.0,165848.0,175433.0,174441.0,159493.0,162214.0,206568.0
4,Farms,88136.0,79030.0,70934.0,76044.0,78093.0,74323.0,91751.0,120237.0,105576.0,97495.0,...,147976.0,183261.0,165998.0,145229.0,129892.0,138885.0,135853.0,119835.0,120732.0,164706.0
5,"Forestry, fishing, and related activities",20501.0,20726.0,21668.0,22262.0,21737.0,21594.0,22846.0,23586.0,23875.0,28861.0,...,30714.0,31024.0,32914.0,34909.0,35956.0,36548.0,38588.0,39658.0,41482.0,41862.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Information-communications-technology-producing industries\3\,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1. Consists of agriculture, forestry, fishing, and hunting; mining; construction; and manufacturing.",Information-communications-technology-producing industries\3\,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2. Consists of utilities; wholesale trade; retail trade; transportation and warehousing; information; finance, insurance, real estate, rental, and leasing; professional and business services; educational services, health care, and social assistance; arts, entertainment, recreation, accommodation, and food services; and other services, except government.",Information-communications-technology-producing industries\3\,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
def read_int_inputs():
    df = pd.read_excel(
        io=data_config['file_path']+data_config['int_in_fname'],
        sheet_name='TII105-A',
        header=[7],
        index_col=[0,1]
    )
    df.drop(columns=df.columns[0], axis=1, inplace=True)
    return df

In [16]:
test_int_in = read_int_inputs()
test_int_in

,,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Line,,,,,,,,,,,,,,,,,,,,,,
1,All industries,6815765.0,7153967.0,7639551.0,8374271.0,8299226.0,8241723.0,8681571.0,9471710.0,10475540.0,11109074.0,...,12968848.0,13506920.0,14205732.0,13977047.0,14159952.0,14914756.0,15956475.0,16328725.0,15501502.0,18089683.0
2,Private industries,6296178.0,6615520.0,7057601.0,7746610.0,7612231.0,7500517.0,7891369.0,8624076.0,9563218.0,10124386.0,...,11739117.0,12292410.0,12984706.0,12749619.0,12894395.0,13615563.0,14566167.0,14841546.0,14032922.0,16434981.0
3,"Agriculture, forestry, fishing, and hunting",149245.0,150007.0,152713.0,145291.0,151526.0,145956.0,154702.0,164732.0,170736.0,177679.0,...,272380.0,271960.0,298505.0,276295.0,266405.0,276952.0,276888.0,279528.0,285565.0,319476.0
4,Farms,131244.0,128792.0,129015.0,128269.0,134241.0,128040.0,136122.0,142940.0,147975.0,158748.0,...,256191.0,253671.0,275668.0,257679.0,249785.0,256494.0,259249.0,268055.0,273663.0,301930.0
5,"Forestry, fishing, and related activities",18001.0,21215.0,23698.0,17021.0,17285.0,17916.0,18580.0,21792.0,22761.0,18931.0,...,16188.0,18287.0,22836.0,18617.0,16620.0,20458.0,17640.0,11474.0,11903.0,17546.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Information-communications-technology-producing industries\3\,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1. Consists of agriculture, forestry, fishing, and hunting; mining; construction; and manufacturing.",Information-communications-technology-producing industries\3\,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"2. Consists of utilities; wholesale trade; retail trade; transportation and warehousing; information; finance, insurance, real estate, rental, and leasing; professional and business services; educational services, health care, and social assistance; arts, entertainment, recreation, accommodation, and food services; and other services, except government.",Information-communications-technology-producing industries\3\,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
def read_pce_bridge(year):
    df = pd.read_excel(
        io=data_config['file_path']+data_config['pce_fname'],
        sheet_name=str(year),
        header=[4]
    )
    return df
    
def read_pce_bridge_all(time_config):
    # read all domestict comm use matrix for all years in time config
    pce_dfs = {}
    for time in time_config:
        df = read_pce_bridge(time)
        pce_dfs[time] = df
    return pce_dfs

In [18]:
test_pce = read_pce_bridge(2021)
test_pce

,NIPA Line,PCE Category,Commodity Code,Commodity Description,Unnamed: 4,Unnamed: 5,Wholesale,Retail,Unnamed: 8,Year
0,5,New motor vehicles,3361MV,"Motor vehicles, bodies and trailers, and parts",266942,4284,5851,74165,351242,2021
1,6,Net purchases of used motor vehicles,Used,"Scrap, used and secondhand goods",156903,2514,3168,80455,243040,2021
2,7,Motor vehicles parts and accessories,327,Nonmetallic mineral products,162,29,24,126,342,2021
3,7,Motor vehicles parts and accessories,331,Primary metals,19,1,2,15,36,2021
4,7,Motor vehicles parts and accessories,332,Fabricated metal products,752,25,320,583,1680,2021
...,...,...,...,...,...,...,...,...,...,...
294,111,Final consumption expenditures of nonprofit in...,623,Nursing and residential care facilities,22282,0,0,0,22282,2021
295,111,Final consumption expenditures of nonprofit in...,624,Social assistance,94434,0,0,0,94434,2021
296,111,Final consumption expenditures of nonprofit in...,711AS,"Performing arts, spectator sports, museums, an...",17624,0,0,0,17624,2021
297,111,Final consumption expenditures of nonprofit in...,713,"Amusements, gambling, and recreation industries",7954,0,0,0,7954,2021


# Make mapping for commodities and industry codes

In [19]:
def infer_comm_map(year):
    # infers commodity code using mkt share data for given year
    df = read_mkt_share(year)
    comm_map = {}
    for pair in df.columns:
        comm_map[pair[0]] = pair[1]
    if len(comm_map) != 73:
        raise ValueError('Expected 73 commodities from inference. Has this changed from BEA?')
    return comm_map

def infer_ind_map(year):
    # infers industry code using mkt share data for given year
    df = read_mkt_share(year)
    ind_map = {}
    for pair in df.index:
        ind_map[pair[0]] = pair[1]
    if len(ind_map) != 71:
        raise ValueError('Expected 71 industries from inference. Has this changed from BEA?')
    return ind_map

In [20]:
comm_map = infer_comm_map(2021)
ind_map = infer_ind_map(2021)

In [21]:
comm_map

{'111CA': 'Farms',
 '113FF': 'Forestry, fishing, and related activities',
 '211': 'Oil and gas extraction',
 '212': 'Mining, except oil and gas',
 '213': 'Support activities for mining',
 '22': 'Utilities',
 '23': 'Construction',
 '321': 'Wood products',
 '327': 'Nonmetallic mineral products',
 '331': 'Primary metals',
 '332': 'Fabricated metal products',
 '333': 'Machinery',
 '334': 'Computer and electronic products',
 '335': 'Electrical equipment, appliances, and components',
 '3361MV': 'Motor vehicles, bodies and trailers, and parts',
 '3364OT': 'Other transportation equipment',
 '337': 'Furniture and related products',
 '339': 'Miscellaneous manufacturing',
 '311FT': 'Food and beverage and tobacco products',
 '313TT': 'Textile mills and textile product mills',
 '315AL': 'Apparel and leather and allied products',
 '322': 'Paper products',
 '323': 'Printing and related support activities',
 '324': 'Petroleum and coal products',
 '325': 'Chemical products',
 '326': 'Plastics and rubbe

In [22]:
ind_map

{'111CA': 'Farms',
 '113FF': 'Forestry, fishing, and related activities',
 '211': 'Oil and gas extraction',
 '212': 'Mining, except oil and gas',
 '213': 'Support activities for mining',
 '22': 'Utilities',
 '23': 'Construction',
 '321': 'Wood products',
 '327': 'Nonmetallic mineral products',
 '331': 'Primary metals',
 '332': 'Fabricated metal products',
 '333': 'Machinery',
 '334': 'Computer and electronic products',
 '335': 'Electrical equipment, appliances, and components',
 '3361MV': 'Motor vehicles, bodies and trailers, and parts',
 '3364OT': 'Other transportation equipment',
 '337': 'Furniture and related products',
 '339': 'Miscellaneous manufacturing',
 '311FT': 'Food and beverage and tobacco products',
 '313TT': 'Textile mills and textile product mills',
 '315AL': 'Apparel and leather and allied products',
 '322': 'Paper products',
 '323': 'Printing and related support activities',
 '324': 'Petroleum and coal products',
 '325': 'Chemical products',
 '326': 'Plastics and rubbe

# Clean data

In [23]:
def clean_mkt_share(in_mkt_share,comm_map,ind_map):
    mkt_share_df = in_mkt_share.copy()
    # filter commodities
    mkt_share_df = mkt_share_df.loc[:,mkt_share_df.columns.get_level_values(0).isin(comm_map)]
    # filter industries
    mkt_share_df = mkt_share_df.loc[mkt_share_df.index.get_level_values(0).isin(ind_map),:]
    mkt_share_df.sort_index(inplace=True) # sort indices
    mkt_share_df = mkt_share_df.reindex(sorted(mkt_share_df.columns), axis=1) # sort columns
    return mkt_share_df

In [24]:
test_mkt_share_clean = clean_mkt_share(test_mkt_share,comm_map,ind_map)
test_mkt_share_clean

,Industries/Commodities,111CA,113FF,211,212,213,22,23,311FT,313TT,315AL,...,81,GFE,GFGD,GFGN,GSLE,GSLG,HS,ORE,Other,Used
,Industry Description,Farms,"Forestry, fishing, and related activities",Oil and gas extraction,"Mining, except oil and gas",Support activities for mining,Utilities,Construction,Food and beverage and tobacco products,Textile mills and textile product mills,Apparel and leather and allied products,...,"Other services, except government",Federal government enterprises,Federal general government (defense),Federal general government (nondefense),State and local government enterprises,State and local general government,Housing,Other real estate,Noncomparable imports and rest-of-the-world adjustment,"Scrap, used and secondhand goods"
111CA,Farms,0.999560,0.067274,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000
113FF,"Forestry, fishing, and related activities",0.000076,0.881106,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000
211,Oil and gas extraction,0.000000,0.000000,0.999369,0.000880,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000
212,"Mining, except oil and gas",0.000000,0.000000,0.000000,0.987722,0.001713,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000
213,Support activities for mining,0.000000,0.000000,0.000000,0.000856,0.998277,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GFGN,Federal general government (nondefense),0.000000,0.000000,0.000000,0.000000,0.000000,0.000098,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,1.0,0.000000,0.0,0.00000,0.000000,1.0,0.000000
GSLE,State and local government enterprises,0.000000,0.000000,0.000000,0.000000,0.000000,0.192990,0.0,0.0,0.0,0.0,...,0.003278,0.0,0.0,0.0,0.981442,0.0,0.00366,0.017335,0.0,0.000000
GSLG,State and local general government,0.000364,0.027821,0.000000,0.000000,0.000000,0.001549,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,1.0,0.00000,0.000000,0.0,0.478292


In [25]:
def clean_dom_use(in_dom_share,comm_map,ind_map):
    dom_use_df = in_dom_share.copy()
    # filter commodities
    dom_use_df = dom_use_df.loc[dom_use_df.index.get_level_values(0).isin(comm_map)]
    # filter industries
    dom_use_df = dom_use_df.loc[:,dom_use_df.columns.get_level_values(0).isin(ind_map)]
    dom_use_df.sort_index(inplace=True)  # sort indices
    dom_use_df = dom_use_df.reindex(sorted(dom_use_df.columns), axis=1) # sort columns
    return dom_use_df

In [26]:
test_dom_use_clean = clean_dom_use(test_dom_use,comm_map,ind_map)
test_dom_use_clean

,Commodities/Industries,111CA,113FF,211,212,213,22,23,311FT,313TT,315AL,...,721,722,81,GFE,GFGD,GFGN,GSLE,GSLG,HS,ORE
,Commodity Description,Farms,"Forestry, fishing, and related activities",Oil and gas extraction,"Mining, except oil and gas",Support activities for mining,Utilities,Construction,Food and beverage and tobacco products,Textile mills and textile product mills,Apparel and leather and allied products,...,Accommodation,Food services and drinking places,"Other services, except government",Federal government enterprises,Federal general government (defense),Federal general government (nondefense),State and local government enterprises,State and local general government,Housing,Other real estate
111CA,Farms,0.173553,0.003526,0.000000,0.000511,0.000208,0.000000,0.001356,2.191741e-01,0.035267,0.000000,...,0.000155,0.000698,0.000014,0.000000,0.000003,0.000573,0.000000,0.000700,0.000001,0.000001
113FF,"Forestry, fishing, and related activities",0.052062,0.144284,0.000000,0.000741,0.000000,0.000000,0.000000,4.764000e-04,0.000000,0.008713,...,0.000144,0.000329,0.000011,0.000013,0.000000,0.000000,0.000000,0.001209,0.000000,0.000000
211,Oil and gas extraction,0.000000,0.000000,0.103113,0.000083,0.000009,0.044571,0.000000,6.000000e-07,0.000000,0.000000,...,0.000157,0.000077,0.000005,0.006025,0.000000,0.000000,0.044277,0.000000,0.000000,0.000000
212,"Mining, except oil and gas",0.003139,0.000005,0.000128,0.079912,0.000344,0.009573,0.010456,2.308000e-04,0.000351,0.000000,...,0.000275,0.000225,0.000764,0.000000,0.000000,0.001053,0.008645,0.000330,0.000189,0.000000
213,Support activities for mining,0.000000,0.000000,0.020004,0.018675,0.010956,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSLG,State and local general government,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
HS,Housing,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ORE,Other real estate,0.059868,0.001112,0.004134,0.000767,0.000831,0.009790,0.012780,2.730400e-03,0.008933,0.003650,...,0.007600,0.081735,0.047730,0.012226,0.000205,0.010466,0.010350,0.010790,0.007730,0.087659


In [27]:
def clean_gross_output(input_gross_data,comm_map,ind_map):
    gross_df = input_gross_data.copy()
    gross_df = gross_df.dropna(how='all') # drop rows where all values are na
    
    index_1_val = gross_df.index.get_level_values(1).str.strip()
    index_1_val = index_1_val.tolist()
    
    rev_ind_map = {}
    for k,v in ind_map.items():
        rev_ind_map[v] = k
        
    new_index = []
    
    prefix = 'Federal'
    for ind in range(len(index_1_val)):
        ind_val = index_1_val[ind]
        
        if (ind_val == 'National defense'):
            ind_val = 'Federal general government (defense)'
        elif (ind_val == 'Nondefense'):
            ind_val = 'Federal general government (nondefense)'
        elif (ind_val == 'Government enterprises'):
            ind_val = prefix + ' government enterprises'
            prefix = 'State and local'
        elif (ind_val == 'General government'):
            ind_val = prefix + ' general government'
        
        if ind_val in rev_ind_map:
            new_index.append((rev_ind_map[ind_val],ind_val))
        else:
            new_index.append(('NA',ind_val))
            
    gross_df = gross_df.set_index(pd.MultiIndex.from_tuples(new_index, names=gross_df.index.names))
    gross_df = gross_df.loc[gross_df.index.get_level_values(1).isin(ind_map.values())]

    return gross_df

In [28]:
test_gross_clean = clean_gross_output(test_gross,comm_map,ind_map)
test_gross_clean

,,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Line,,,,,,,,,,,,,,,,,,,,,,
111CA,Farms,219380.0,207822.0,199949.0,204313.0,212334.0,202363.0,227873.0,263177.0,253552.0,256245.0,...,404166.0,436934.0,441667.0,402908.0,379677.0,395379.0,395102.0,387890.0,394395.0,466636.0
113FF,"Forestry, fishing, and related activities",38502.0,41942.0,45365.0,39283.0,39022.0,39509.0,41426.0,45377.0,46636.0,47792.0,...,46902.0,49312.0,55750.0,53526.0,52575.0,57006.0,56228.0,51132.0,53384.0,59408.0
211,Oil and gas extraction,98509.0,72745.0,85524.0,142988.0,142309.0,116721.0,165982.0,191027.0,249252.0,257413.0,...,362652.0,426372.0,476039.0,268953.0,230486.0,305097.0,410045.0,375173.0,255273.0,455730.0
212,"Mining, except oil and gas",56243.0,55373.0,53050.0,54676.0,55094.0,54442.0,57394.0,67311.0,84276.0,100825.0,...,124665.0,114553.0,117311.0,106189.0,96130.0,105504.0,111313.0,111909.0,98029.0,108964.0
213,Support activities for mining,22119.0,23419.0,17543.0,20888.0,29205.0,24831.0,29534.0,41728.0,60961.0,86922.0,...,126168.0,122511.0,152640.0,120830.0,61795.0,87745.0,105817.0,107732.0,63020.0,63819.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GFGD,Federal general government (defense),304866.0,300962.0,313270.0,321572.0,339792.0,374582.0,428318.0,466347.0,497159.0,522568.0,...,676302.0,637602.0,625335.0,613306.0,614739.0,626891.0,663650.0,707491.0,731008.0,744145.0
GFGN,Federal general government (nondefense),160267.0,163523.0,168137.0,182355.0,196175.0,217788.0,236785.0,252026.0,267806.0,283017.0,...,370376.0,366849.0,375908.0,391290.0,402490.0,407860.0,428185.0,444868.0,502087.0,572449.0
GFE,Federal government enterprises,76409.0,77645.0,81071.0,83591.0,83636.0,87238.0,86948.0,87641.0,88084.0,95384.0,...,97146.0,93199.0,94524.0,94062.0,93794.0,91422.0,92031.0,90680.0,88764.0,92356.0


In [29]:
def clean_delta_gross_output(input_delta_gross_data,comm_map,ind_map):
    gross_df = input_delta_gross_data.copy()
    gross_df = gross_df.dropna(how='all') # drop rows where all values are na
    
    index_1_val = gross_df.index.get_level_values(1).str.strip()
    index_1_val = index_1_val.tolist()
    
    rev_ind_map = {}
    for k,v in ind_map.items():
        rev_ind_map[v] = k
        
    new_index = []
    
    prefix = 'Federal'
    for ind in range(len(index_1_val)):
        ind_val = index_1_val[ind]
        
        if (ind_val == 'National defense'):
            ind_val = 'Federal general government (defense)'
        elif (ind_val == 'Nondefense'):
            ind_val = 'Federal general government (nondefense)'
        elif (ind_val == 'Government enterprises'):
            ind_val = prefix + ' government enterprises'
            prefix = 'State and local'
        elif (ind_val == 'General government'):
            ind_val = prefix + ' general government'
        
        if ind_val in rev_ind_map:
            new_index.append((rev_ind_map[ind_val],ind_val))
        else:
            new_index.append(('NA',ind_val))
            
    gross_df = gross_df.set_index(pd.MultiIndex.from_tuples(new_index, names=gross_df.index.names))
    gross_df = gross_df.loc[gross_df.index.get_level_values(1).isin(ind_map.values())]

    return gross_df

In [30]:
test_delta_gross

1998  \
Line                                                                                                          
1                                                      All industries                                   0.0   
2                                                  Private industries                                  -0.2   
3                                                    Agriculture, forestry, fishing, and hunting       -5.2   
4                                                      Farms                                           -5.8   
5                                                      Forestry, fishing, and related activities       -1.6   
...                                                                                                     ...   
100                                                      Information-communications-technology-pro...  -7.2   
                                                         Information-communications-technology-pro...   NaN   
1. Consists of agriculture, forestry, fishing, ...       Information-communications-technology-pro...   NaN   
2. Consists of utilities; wholesale trade; reta...       Information-communications-technology-pro...   NaN   
3. Consists of computer and electronic product ...       Information-communications-technology-pro...   NaN   

                                                                                                       1999  \
Line                                                                                                          
1                                                      All industries                                   1.3   
2                                                  Private industries                                   1.1   
3                                                    Agriculture, forestry, fishing, and hunting       -5.0   
4                                                      Farms                                           -5.9   
5                                                      Forestry, fishing, and related activities       -0.8   
...                                                                                                     ...   
100                                                      Information-communications-technology-pro...  -5.4   
                                                         Information-communications-technology-pro...   NaN   
1. Consists of agriculture, forestry, fishing, ...       Information-communications-technology-pro...   NaN   
2. Consists of utilities; wholesale trade; reta...       Information-communications-technology-pro...   NaN   
3. Consists of computer and electronic product ...       Information-communications-technology-pro...   NaN   

                                                                                                       2000  \
Line                                                                                                          
1                                                      All industries                                   3.2   
2                                                  Private industries                                   3.1   
3                                                    Agriculture, forestry, fishing, and hunting        0.7   
4                                                      Farms                                            0.9   
5                                                      Forestry, fishing, and related activities        0.1   
...                                                                                                     ...   
100                                                      Information-communications-technology-pro...  -4.2   
                                                         Information-communications-technology-pro...   NaN   
1. Consists of agriculture, forestry, fishing, ...       Information-communications-technology-pro...   N

In [31]:
test_delta_gross_clean = clean_delta_gross_output(test_delta_gross,comm_map,ind_map)
test_delta_gross_clean

,,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Line,,,,,,,,,,,,,,,,,,,,,,
111CA,Farms,-5.8,-5.9,0.9,5.3,-4.5,9.2,12.2,-4.5,0.4,18.5,...,4.9,1.8,-1.1,-10.9,-11.1,2.7,0.0,-0.4,-2.7,23.0
113FF,"Forestry, fishing, and related activities",-1.6,-0.8,0.1,-7.9,-1.1,2.3,5.2,3.7,3.4,6.0,...,1.4,3.3,2.9,-0.6,0.3,1.7,2.1,0.2,-0.2,5.3
211,Oil and gas extraction,-24.2,20.1,60.7,-4.1,-12.5,42.0,19.3,31.4,2.5,6.2,...,-12.4,6.7,-1.1,-42.3,-11.0,22.7,15.9,-14.9,-26.7,85.8
212,"Mining, except oil and gas",-3.4,-1.8,1.6,1.7,3.7,4.9,13.4,18.3,13.0,7.5,...,-0.3,-5.0,-1.9,-6.4,-1.4,8.3,5.1,0.2,-0.3,9.6
213,Support activities for mining,5.0,-9.0,-4.9,-2.8,0.5,2.5,4.0,8.9,6.6,3.8,...,4.3,1.8,1.6,0.1,-1.9,2.1,-2.0,1.4,-3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GFGD,Federal general government (defense),1.5,2.6,3.9,2.5,4.3,5.9,4.0,5.2,4.0,3.4,...,1.1,0.7,1.5,0.3,0.5,1.7,3.1,1.8,1.2,3.9
GFGN,Federal general government (nondefense),1.6,3.1,3.9,2.2,4.3,4.4,4.0,4.0,3.1,3.2,...,0.7,1.7,2.3,1.1,1.1,2.4,3.5,2.2,1.1,3.2
GFE,Federal government enterprises,0.0,2.2,0.5,5.1,3.4,2.6,0.4,0.4,5.8,3.9,...,1.2,1.2,4.1,1.4,-1.8,0.8,1.5,1.3,1.1,4.9


In [32]:
def clean_value_add(value_add,comm_map,ind_map):
    value_df = value_add.copy()
    value_df = value_df.dropna(how='all') # drop rows where all values are na
    
    index_1_val = value_df.index.get_level_values(1).str.strip()
    index_1_val = index_1_val.tolist()
    
    rev_ind_map = {}
    for k,v in ind_map.items():
        rev_ind_map[v] = k
        
    new_index = []
    
    prefix = 'Federal'
    for ind in range(len(index_1_val)):
        ind_val = index_1_val[ind]
        
        if (ind_val == 'National defense'):
            ind_val = 'Federal general government (defense)'
        elif (ind_val == 'Nondefense'):
            ind_val = 'Federal general government (nondefense)'
        elif (ind_val == 'Government enterprises'):
            ind_val = prefix + ' government enterprises'
            prefix = 'State and local'
        elif (ind_val == 'General government'):
            ind_val = prefix + ' general government'
        
        if ind_val in rev_ind_map:
            new_index.append((rev_ind_map[ind_val],ind_val))
        else:
            new_index.append(('NA',ind_val))
            
    value_df = value_df.set_index(pd.MultiIndex.from_tuples(new_index, names=value_df.index.names))
    value_df = value_df.loc[value_df.index.get_level_values(1).isin(ind_map.values())]

    return value_df

In [33]:
test_val_add_clean = clean_value_add(test_val_add,comm_map,ind_map)
test_val_add_clean

,,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Line,,,,,,,,,,,,,,,,,,,,,,
111CA,Farms,88136.0,79030.0,70934.0,76044.0,78093.0,74323.0,91751.0,120237.0,105576.0,97495.0,...,147976.0,183261.0,165998.0,145229.0,129892.0,138885.0,135853.0,119835.0,120732.0,164706.0
113FF,"Forestry, fishing, and related activities",20501.0,20726.0,21668.0,22262.0,21737.0,21594.0,22846.0,23586.0,23875.0,28861.0,...,30714.0,31024.0,32914.0,34909.0,35956.0,36548.0,38588.0,39658.0,41482.0,41862.0
211,Oil and gas extraction,55060.0,39065.0,44086.0,68211.0,76100.0,69862.0,92853.0,112910.0,155789.0,178743.0,...,224919.0,255989.0,278733.0,158122.0,138565.0,175227.0,206494.0,187888.0,110943.0,239216.0
212,"Mining, except oil and gas",26501.0,29004.0,29358.0,28506.0,28053.0,27315.0,29239.0,33614.0,42845.0,51799.0,...,66986.0,62339.0,58851.0,50018.0,47408.0,56813.0,59285.0,57282.0,56579.0,61242.0
213,Support activities for mining,13500.0,13622.0,11141.0,13824.0,19732.0,15208.0,16853.0,19904.0,26781.0,42604.0,...,68619.0,69426.0,79458.0,53558.0,32107.0,43357.0,55777.0,53513.0,33540.0,33477.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GFGD,Federal general government (defense),211818.0,210429.0,212016.0,217605.0,223004.0,239722.0,264264.0,280992.0,301197.0,315073.0,...,407611.0,400101.0,400046.0,398359.0,399112.0,403416.0,417571.0,431083.0,448411.0,469774.0
GFGN,Federal general government (nondefense),121703.0,126356.0,132953.0,142711.0,147260.0,158122.0,170416.0,178379.0,187155.0,194818.0,...,256053.0,258307.0,266800.0,275298.0,284773.0,294423.0,307090.0,317058.0,333934.0,349566.0
GFE,Federal government enterprises,57206.0,59647.0,61029.0,63219.0,60155.0,64644.0,64200.0,66142.0,62922.0,65657.0,...,50223.0,45398.0,48981.0,57621.0,59120.0,62166.0,63131.0,61696.0,67050.0,68172.0


In [34]:
def clean_int_inputs(int_inputs,comm_map,ind_map):
    int_df = int_inputs.copy()
    int_df = int_df.dropna(how='all') # drop rows where all values are na
    
    index_1_val = int_df.index.get_level_values(1).str.strip()
    index_1_val = index_1_val.tolist()
    
    rev_ind_map = {}
    for k,v in ind_map.items():
        rev_ind_map[v] = k
        
    new_index = []
    
    prefix = 'Federal'
    for ind in range(len(index_1_val)):
        ind_val = index_1_val[ind]
        
        if (ind_val == 'National defense'):
            ind_val = 'Federal general government (defense)'
        elif (ind_val == 'Nondefense'):
            ind_val = 'Federal general government (nondefense)'
        elif (ind_val == 'Government enterprises'):
            ind_val = prefix + ' government enterprises'
            prefix = 'State and local'
        elif (ind_val == 'General government'):
            ind_val = prefix + ' general government'
        
        if ind_val in rev_ind_map:
            new_index.append((rev_ind_map[ind_val],ind_val))
        else:
            new_index.append(('NA',ind_val))
            
    int_df = int_df.set_index(pd.MultiIndex.from_tuples(new_index, names=int_df.index.names))
    int_df = int_df.loc[int_df.index.get_level_values(1).isin(ind_map.values())]

    return int_df

In [35]:
test_int_in_clean = clean_int_inputs(test_int_in,comm_map,ind_map)
test_int_in_clean

,,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Line,,,,,,,,,,,,,,,,,,,,,,
111CA,Farms,131244.0,128792.0,129015.0,128269.0,134241.0,128040.0,136122.0,142940.0,147975.0,158748.0,...,256191.0,253671.0,275668.0,257679.0,249785.0,256494.0,259249.0,268055.0,273663.0,301930.0
113FF,"Forestry, fishing, and related activities",18001.0,21215.0,23698.0,17021.0,17285.0,17916.0,18580.0,21792.0,22761.0,18931.0,...,16188.0,18287.0,22836.0,18617.0,16620.0,20458.0,17640.0,11474.0,11903.0,17546.0
211,Oil and gas extraction,43449.0,33680.0,41438.0,74777.0,66209.0,46859.0,73129.0,78116.0,93463.0,78670.0,...,137733.0,170384.0,197306.0,110831.0,91922.0,129870.0,203551.0,187285.0,144330.0,216514.0
212,"Mining, except oil and gas",29742.0,26369.0,23692.0,26170.0,27042.0,27127.0,28155.0,33696.0,41431.0,49026.0,...,57679.0,52215.0,58459.0,56171.0,48722.0,48691.0,52028.0,54627.0,41451.0,47722.0
213,Support activities for mining,8619.0,9797.0,6402.0,7064.0,9473.0,9623.0,12681.0,21824.0,34179.0,44318.0,...,57549.0,53085.0,73182.0,67272.0,29688.0,44388.0,50040.0,54219.0,29481.0,30342.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GFGD,Federal general government (defense),93048.0,90533.0,101254.0,103967.0,116789.0,134860.0,164055.0,185356.0,195963.0,207497.0,...,268691.0,237500.0,225289.0,214946.0,215627.0,223475.0,246079.0,276408.0,282597.0,274370.0
GFGN,Federal general government (nondefense),38564.0,37168.0,35185.0,39644.0,48914.0,59666.0,66369.0,73647.0,80651.0,88199.0,...,114324.0,108542.0,109107.0,115992.0,117717.0,113437.0,121094.0,127810.0,168153.0,222883.0
GFE,Federal government enterprises,19204.0,17999.0,20042.0,20372.0,23481.0,22594.0,22747.0,21499.0,25161.0,29727.0,...,46922.0,47801.0,45543.0,36441.0,34674.0,29256.0,28900.0,28984.0,21714.0,24184.0


In [36]:
def clean_pce_bridge(in_pce,comm_map):
    pce_df = in_pce.copy()
    for comm in pce_df['Commodity Code']:
        if str(comm) not in comm_map:
            print(f'{comm} not in comm')
    pce_df.rename(
        columns = {
            'Unnamed: 4': "Producers' Value",
            'Unnamed: 5': 'Transportation Costs',
            'Unnamed: 8': "Purchasers' Value"
        },
        inplace=True
    )
    return pce_df
                

In [37]:
test_pce_clean = clean_pce_bridge(test_pce,comm_map)
test_pce_clean

,NIPA Line,PCE Category,Commodity Code,Commodity Description,Producers' Value,Transportation Costs,Wholesale,Retail,Purchasers' Value,Year
0,5,New motor vehicles,3361MV,"Motor vehicles, bodies and trailers, and parts",266942,4284,5851,74165,351242,2021
1,6,Net purchases of used motor vehicles,Used,"Scrap, used and secondhand goods",156903,2514,3168,80455,243040,2021
2,7,Motor vehicles parts and accessories,327,Nonmetallic mineral products,162,29,24,126,342,2021
3,7,Motor vehicles parts and accessories,331,Primary metals,19,1,2,15,36,2021
4,7,Motor vehicles parts and accessories,332,Fabricated metal products,752,25,320,583,1680,2021
...,...,...,...,...,...,...,...,...,...,...
294,111,Final consumption expenditures of nonprofit in...,623,Nursing and residential care facilities,22282,0,0,0,22282,2021
295,111,Final consumption expenditures of nonprofit in...,624,Social assistance,94434,0,0,0,94434,2021
296,111,Final consumption expenditures of nonprofit in...,711AS,"Performing arts, spectator sports, museums, an...",17624,0,0,0,17624,2021
297,111,Final consumption expenditures of nonprofit in...,713,"Amusements, gambling, and recreation industries",7954,0,0,0,7954,2021


In [38]:
test_mkt_share_clean.to_csv('test_mkt_share.csv')
test_dom_use_clean.to_csv('test_dom_use.csv')
test_gross_clean.to_csv('test_gross.csv')
test_delta_gross_clean.to_csv('test_delta_gross.csv')
test_val_add_clean.to_csv('test_val_add.csv')
test_int_in_clean.to_csv('test_int_in.csv')
test_pce_clean.to_csv('test_pce.csv')

In [39]:
ind_map

{'111CA': 'Farms',
 '113FF': 'Forestry, fishing, and related activities',
 '211': 'Oil and gas extraction',
 '212': 'Mining, except oil and gas',
 '213': 'Support activities for mining',
 '22': 'Utilities',
 '23': 'Construction',
 '321': 'Wood products',
 '327': 'Nonmetallic mineral products',
 '331': 'Primary metals',
 '332': 'Fabricated metal products',
 '333': 'Machinery',
 '334': 'Computer and electronic products',
 '335': 'Electrical equipment, appliances, and components',
 '3361MV': 'Motor vehicles, bodies and trailers, and parts',
 '3364OT': 'Other transportation equipment',
 '337': 'Furniture and related products',
 '339': 'Miscellaneous manufacturing',
 '311FT': 'Food and beverage and tobacco products',
 '313TT': 'Textile mills and textile product mills',
 '315AL': 'Apparel and leather and allied products',
 '322': 'Paper products',
 '323': 'Printing and related support activities',
 '324': 'Petroleum and coal products',
 '325': 'Chemical products',
 '326': 'Plastics and rubbe

In [40]:
comm_map

{'111CA': 'Farms',
 '113FF': 'Forestry, fishing, and related activities',
 '211': 'Oil and gas extraction',
 '212': 'Mining, except oil and gas',
 '213': 'Support activities for mining',
 '22': 'Utilities',
 '23': 'Construction',
 '321': 'Wood products',
 '327': 'Nonmetallic mineral products',
 '331': 'Primary metals',
 '332': 'Fabricated metal products',
 '333': 'Machinery',
 '334': 'Computer and electronic products',
 '335': 'Electrical equipment, appliances, and components',
 '3361MV': 'Motor vehicles, bodies and trailers, and parts',
 '3364OT': 'Other transportation equipment',
 '337': 'Furniture and related products',
 '339': 'Miscellaneous manufacturing',
 '311FT': 'Food and beverage and tobacco products',
 '313TT': 'Textile mills and textile product mills',
 '315AL': 'Apparel and leather and allied products',
 '322': 'Paper products',
 '323': 'Printing and related support activities',
 '324': 'Petroleum and coal products',
 '325': 'Chemical products',
 '326': 'Plastics and rubbe

In [41]:
test_mkt_share_clean @ test_dom_use_clean

,Commodities/Industries,111CA,113FF,211,212,213,22,23,311FT,313TT,315AL,...,721,722,81,GFE,GFGD,GFGN,GSLE,GSLG,HS,ORE
,Commodity Description,Farms,"Forestry, fishing, and related activities",Oil and gas extraction,"Mining, except oil and gas",Support activities for mining,Utilities,Construction,Food and beverage and tobacco products,Textile mills and textile product mills,Apparel and leather and allied products,...,Accommodation,Food services and drinking places,"Other services, except government",Federal government enterprises,Federal general government (defense),Federal general government (nondefense),State and local government enterprises,State and local general government,Housing,Other real estate
111CA,Farms,0.176980,1.323094e-02,0.000000,0.000561,2.098268e-04,0.000001,1.357523e-03,2.191102e-01,0.035252,5.863070e-04,...,0.000178,0.000724,1.813401e-05,9.541155e-07,6.218993e-06,5.732522e-04,8.243440e-06,0.000786,1.222080e-06,7.542256e-06
113FF,"Forestry, fishing, and related activities",0.045886,1.271294e-01,0.000000,0.000653,1.582320e-08,0.000000,1.030256e-07,4.364160e-04,0.000003,7.676986e-03,...,0.000127,0.000290,9.781352e-06,1.163060e-05,2.280000e-10,4.357840e-08,0.000000e+00,0.001066,8.360000e-11,9.120000e-11
211,Oil and gas extraction,0.000676,2.743907e-04,0.103978,0.002011,1.060230e-03,0.046347,1.531899e-03,1.688360e-04,0.000136,7.960070e-05,...,0.000404,0.000256,1.952479e-04,7.099893e-03,8.242541e-04,7.995138e-05,4.599185e-02,0.002155,6.331902e-06,2.512042e-04
212,"Mining, except oil and gas",0.003259,6.472927e-05,0.000234,0.079058,4.063215e-04,0.009478,1.047258e-02,2.615661e-04,0.000835,4.525464e-05,...,0.000280,0.000231,7.725918e-04,6.551577e-06,9.406003e-06,1.052363e-03,8.584422e-03,0.000374,1.891692e-04,2.023524e-05
213,Support activities for mining,0.000011,3.853705e-06,0.020011,0.018740,1.095241e-02,0.000011,1.330920e-04,3.845873e-05,0.000039,5.158767e-05,...,0.000011,0.000009,6.786428e-06,1.547881e-05,1.556849e-05,5.877241e-06,3.719611e-05,0.000004,5.273279e-07,1.168815e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GFGN,Federal general government (nondefense),0.000001,1.756032e-07,0.000002,0.000003,9.349450e-08,0.000006,3.633069e-07,7.298053e-07,0.000002,5.142002e-07,...,0.000003,0.000003,3.933133e-07,9.639723e-07,2.357530e-07,2.341670e-07,7.782560e-07,0.000001,8.321500e-10,8.849044e-06
GSLE,State and local government enterprises,0.003131,5.284684e-04,0.003663,0.007066,2.975368e-04,0.020917,1.017348e-03,2.113165e-03,0.004726,1.546585e-03,...,0.008321,0.008500,2.890068e-03,3.730668e-03,1.168882e-03,1.625312e-03,5.149094e-03,0.005927,1.471793e-04,1.990825e-02
GSLG,State and local general government,0.001508,4.501380e-03,0.000476,0.000306,6.074553e-03,0.004653,9.306976e-04,2.491350e-04,0.000196,4.208160e-04,...,0.000683,0.000484,4.248880e-03,1.362703e-03,1.000448e-03,2.749758e-04,1.254237e-03,0.003173,1.919581e-05,1.601377e-03


In [42]:
def mat_mul_check(a,b):
    # matrix multiplication a times b, checking for consistent inner dimensions
    # check columns of a are same as rows of b
    diff = a.columns.difference(b.index) # multiindex of difference
    if len(diff) != 0:
        raise ValueError(f'columns of a do not match index of b')
    return a @ b

In [43]:
mat_mul_check(test_mkt_share,test_dom_use_clean)

,Commodities/Industries,111CA,113FF,211,212,213,22,23,311FT,313TT,315AL,...,721,722,81,GFE,GFGD,GFGN,GSLE,GSLG,HS,ORE
,Commodity Description,Farms,"Forestry, fishing, and related activities",Oil and gas extraction,"Mining, except oil and gas",Support activities for mining,Utilities,Construction,Food and beverage and tobacco products,Textile mills and textile product mills,Apparel and leather and allied products,...,Accommodation,Food services and drinking places,"Other services, except government",Federal government enterprises,Federal general government (defense),Federal general government (nondefense),State and local government enterprises,State and local general government,Housing,Other real estate
111CA,Farms,0.176980,1.323094e-02,0.000000,0.000561,2.098268e-04,0.000001,1.357523e-03,2.191102e-01,0.035252,5.863070e-04,...,0.000178,0.000724,1.813401e-05,9.541155e-07,6.218993e-06,5.732522e-04,8.243440e-06,0.000786,1.222080e-06,7.542256e-06
113FF,"Forestry, fishing, and related activities",0.045886,1.271294e-01,0.000000,0.000653,1.582320e-08,0.000000,1.030256e-07,4.364160e-04,0.000003,7.676986e-03,...,0.000127,0.000290,9.781352e-06,1.163060e-05,2.280000e-10,4.357840e-08,0.000000e+00,0.001066,8.360000e-11,9.120000e-11
211,Oil and gas extraction,0.000676,2.743907e-04,0.103978,0.002011,1.060230e-03,0.046347,1.531899e-03,1.688360e-04,0.000136,7.960070e-05,...,0.000404,0.000256,1.952479e-04,7.099893e-03,8.242541e-04,7.995138e-05,4.599185e-02,0.002155,6.331902e-06,2.512042e-04
212,"Mining, except oil and gas",0.003259,6.472927e-05,0.000234,0.079058,4.063215e-04,0.009478,1.047258e-02,2.615661e-04,0.000835,4.525464e-05,...,0.000280,0.000231,7.725918e-04,6.551577e-06,9.406003e-06,1.052363e-03,8.584422e-03,0.000374,1.891692e-04,2.023524e-05
213,Support activities for mining,0.000011,3.853705e-06,0.020011,0.018740,1.095241e-02,0.000011,1.330920e-04,3.845873e-05,0.000039,5.158767e-05,...,0.000011,0.000009,6.786428e-06,1.547881e-05,1.556849e-05,5.877241e-06,3.719611e-05,0.000004,5.273279e-07,1.168815e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GFGD,Federal general government (defense),0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
GFGN,Federal general government (nondefense),0.000001,1.756032e-07,0.000002,0.000003,9.349450e-08,0.000006,3.633069e-07,7.298053e-07,0.000002,5.142002e-07,...,0.000003,0.000003,3.933133e-07,9.639723e-07,2.357530e-07,2.341670e-07,7.782560e-07,0.000001,8.321500e-10,8.849044e-06
GFE,Federal government enterprises,0.000254,7.445721e-05,0.000375,0.000640,4.117304e-05,0.001555,7.810312e-05,1.427078e-04,0.000374,1.182899e-04,...,0.003379,0.003010,1.245972e-03,2.139703e-04,4.497425e-05,6.018179e-04,5.134751e-04,0.000719,1.805817e-05,1.986770e-03


# Modeling